In [ ]:
!pip install qiskit matplotlib numpy scipy plotly pandas scikit-learn torch gym stable-baselines3

In [ ]:
import numpy as np
import plotly.graph_objects as go
from qiskit import QuantumCircuit, execute, Aer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def bloch_coordinates(statevector):
    """Convert a statevector to Bloch sphere coordinates."""
    rho = np.outer(statevector, np.conj(statevector))
    x = 2 * rho[0][1].real
    y = 2 * rho[0][1].imag
    z = rho[0][0] - rho[1][1]
    return x, y, z

def plot_bloch_vector_3d(fig, x, y, z):
    """Plot a Bloch vector in 3D."""
    fig.add_trace(go.Scatter3d(x=[0, x], y=[0, y], z=[0, z],
                               mode='lines',
                               line=dict(color='red', width=10)))
    fig.add_trace(go.Scatter3d(x=[x], y=[y], z=[z],
                               mode='markers',
                               marker=dict(size=10, color='red')))

def plot_bloch_sphere_3d():
    """Create an interactive 3D Bloch sphere plot."""
    fig = go.Figure()

    # Create sphere
    u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
    x = np.cos(u)*np.sin(v)
    y = np.sin(u)*np.sin(v)
    z = np.cos(v)
    fig.add_trace(go.Surface(x=x, y=y, z=z, opacity=0.3))

    # Add axes
    axes = ['x', 'y', 'z']
    colors = ['blue', 'green', 'red']
    for i, axis in enumerate(axes):
        fig.add_trace(go.Scatter3d(x=[0, 0] if axis != 'x' else [-1.5, 1.5],
                                   y=[0, 0] if axis != 'y' else [-1.5, 1.5],
                                   z=[0, 0] if axis != 'z' else [-1.5, 1.5],
                                   mode='lines',
                                   line=dict(color=colors[i], width=5)))
        fig.add_annotation(x=1.5 if axis == 'x' else 0,
                           y=1.5 if axis == 'y' else 0,
                           z=1.5 if axis == 'z' else 0,
                           text=axis,
                           showarrow=False)

    fig.update_layout(scene=dict(aspectmode="cube"))
    return fig

In [ ]:
# Create a quantum circuit
qc = QuantumCircuit(1)
qc.h(0)  # Apply Hadamard gate to create superposition
qc.ry(np.pi/4, 0)  # Apply rotation around y-axis

# Simulate the circuit
backend = Aer.get_backend('statevector_simulator')
job = execute(qc, backend)
result = job.result()
statevector = result.get_statevector()

# Get Bloch coordinates
x, y, z = bloch_coordinates(statevector)

# Create 3D Bloch sphere plot
fig = plot_bloch_sphere_3d()
plot_bloch_vector_3d(fig, x, y, z)

# Show the interactive 3D plot
fig.show()

In [ ]:
def evolve_state(initial_state, time):
    """Evolve the state over time (simple rotation around z-axis)."""
    rotation = np.exp(-1j * time)
    return np.array([initial_state[0], initial_state[1] * rotation])

# Create initial state
initial_state = np.array([1/np.sqrt(2), 1/np.sqrt(2)])

# Create figure
fig = plot_bloch_sphere_3d()

# Animate state evolution
frames = []
for t in np.linspace(0, 2*np.pi, 50):
    evolved_state = evolve_state(initial_state, t)
    x, y, z = bloch_coordinates(evolved_state)
    frame = go.Frame(data=[go.Scatter3d(x=[0, x], y=[0, y], z=[0, z],
                                        mode='lines+markers',
                                        line=dict(color='red', width=10),
                                        marker=dict(size=10, color='red'))])
    frames.append(frame)

fig.frames = frames

# Add play and pause buttons
fig.update_layout(updatemenus=[dict(type='buttons',
                                    showactive=False,
                                    buttons=[dict(label='Play',
                                                  method='animate',
                                                  args=[None, dict(frame=dict(duration=50, redraw=True), fromcurrent=True)]),
                                             dict(label='Pause',
                                                  method='animate',
                                                  args=[[None], dict(frame=dict(duration=0, redraw=False), mode='immediate')])])])

fig.show()